In [1]:
import pandas
from loguru import logger
import ndf.download
from ndf.datamining import datamining
import warnings
import sys
import pandas as pd

warnings.simplefilter(action='ignore', category=FutureWarning)

log_level = 'ERROR'
logger.add(sys.stderr, format="{time} {level} {message}", filter="my_module", level=log_level)
# logger.add(sys.stdout, colorize=True, format="<green>{time}</green> <level>{message}</level>")
logger.add("ndfdownload.log", rotation="30 MB", level=log_level)

2

In [2]:
from datetime import datetime

a = datetime.now()
a.date()

datetime.date(2023, 8, 18)

In [3]:
logger.remove()

In [4]:
d = ndf.download.download()
d.download_all()

C:\Users\souzacle\PycharmProjects\ndfutil\venv\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.traditionsef.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\souzacle\PycharmProjects\ndfutil\venv\lib\site-packages\scrapy\selector\unified.py:83: UserWarning: Selector got both text and root, root is being ignored.
  super().__init__(text=text, type=st, root=root, **kwargs)


bgc ok
tradition ok
prebontullet ok
gfi ok


In [5]:
mining = datamining()

In [6]:
tulletprebon = mining.tulletprebon_calcs()
tulletprebon

,Total for human,Volume
Class,,
BMF1,871.1M,871100000
BMF2,130M,130000000
BROKEN,278.5M,278500000
GT 1 YEAR,70M,70000000
LONGER,130M,130000000
PTAX,120.5M,120500000
TOMPTAX,62.5M,62500000
TOTAL,1.96B,1956700000
XP,294.1M,294100000


In [7]:
bgc = mining.bgc_calcs()
bgc

,Total for human,Volume
Class,,
BMF,129M,129000000
BROKEN,128.84M,128842400
TOTAL,257.84M,257842400


In [8]:
tradition = mining.tradition_calcs()
tradition

,Total for human,Volume
Class,,
BMF,2.5M,2500000
BROKEN,2.5M,2500000
TOTAL,5M,5000000


In [9]:
gfi = mining.gfi_calcs()
gfi

,Total for human,Volume
Class,,
BMF,215M,215000000
BROKEN,65M,65000000
TOTAL,280M,280000000


In [10]:
tulletprebon['source'] = 'TulletPrebon'
tradition['source'] = 'Tradition'
bgc['source'] = 'bgc'
gfi['source'] = 'gfi'

In [11]:
tradition

,Total for human,Volume,source
Class,,,
BMF,2.5M,2500000,Tradition
BROKEN,2.5M,2500000,Tradition
TOTAL,5M,5000000,Tradition


In [12]:
pd.DataFrame({'Class': ['TOTAL'], 'Total for human': [0], 'Volume': [0]}).set_index('Class')

,Total for human,Volume
Class,,
TOTAL,0,0


In [13]:
df_result = pd.concat([tulletprebon, tradition, bgc, gfi])
df_result

,Total for human,Volume,source
Class,,,
BMF1,871.1M,871100000,TulletPrebon
BMF2,130M,130000000,TulletPrebon
BROKEN,278.5M,278500000,TulletPrebon
GT 1 YEAR,70M,70000000,TulletPrebon
LONGER,130M,130000000,TulletPrebon
PTAX,120.5M,120500000,TulletPrebon
TOMPTAX,62.5M,62500000,TulletPrebon
TOTAL,1.96B,1956700000,TulletPrebon
XP,294.1M,294100000,TulletPrebon


In [14]:
df_totals = df_result.query('Class == "TOTAL"')
df_totals

,Total for human,Volume,source
Class,,,
TOTAL,1.96B,1956700000,TulletPrebon
TOTAL,5M,5000000,Tradition
TOTAL,257.84M,257842400,bgc
TOTAL,280M,280000000,gfi


In [15]:
df_totals.groupby(['Class']).sum()

,Volume
Class,
TOTAL,2499542400


In [16]:
from numerize import numerize

market_total = df_totals.sum()['Volume']
market_total = numerize.numerize(float(market_total))
market_total

'2.5B'

In [17]:
summary = mining.market_summary()
summary

,Total for human,Volume,source
Class,,,
TOTAL,1.96B,1956700000,TulletPrebon
TOTAL,5M,5000000,Tradition
TOTAL,257.84M,257842400,bgc
TOTAL,280M,280000000,gfi


In [18]:
market_total = mining.market_total()
market_total

'2.5B'

In [19]:
a = summary[['source', 'Total for human']]
a

,source,Total for human
Class,,
TOTAL,TulletPrebon,1.96B
TOTAL,Tradition,5M
TOTAL,bgc,257.84M
TOTAL,gfi,280M


In [20]:
labels = summary[['source', 'Total for human']]
labels = labels.values.tolist()
labels_chart = []
for item in labels_chart:
        labels_chart.append(f'{item[0]} ${item[1]}')
labels_chart

[]

In [21]:
import matplotlib.pyplot as plt

market_total = mining.market_total()
summary = mining.market_summary()

# Labels prepare
labels = summary[['source', 'Total for human']]
labels = labels.values.tolist()
labels_chart = []
for item in list:
        labels_chart.append(f'{item[0]} ${item[1]}')

font1 = {'family':'serif','color':'blue','size':18}
colors = ['#045ca3', '#f5b12b', '#d11e20', '#8ec5f2']
fig = plt.figure(figsize=(5, 6))
plt.pie(summary['Volume'], labels=labels_chart, labeldistance=1.15, autopct='%1.0f%%',
        wedgeprops = { 'linewidth' : 3, 'edgecolor' : 'white' }, colors=colors,
        textprops={'fontsize': 14})
plt.legend(labels_chart, loc='lower left', bbox_to_anchor=(-0.35, .5), fontsize=10)
plt.title(f'NDF Market Summary\n Total ${market_total}', fontdict=font1)
plt.savefig('pie.png', dpi=fig.dpi, bbox_inches='tight')


TypeError: 'type' object is not iterable

In [ ]:
print(plt.savefig('pie.png', dpi=fig.dpi, bbox_inches='tight'))

In [ ]:
mining.market_total()

In [ ]:
mining.market_summary()

In [ ]:
mining.generate_chart()